In [ ]:
%matplotlib inline
import os
import sys
import gc
from tqdm.notebook import tqdm

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from commons.configuration_manager import ConfigurationManager
from src.learning.training.label_collector import LabelCollector
from src.learning.training.training_file_reader import TrainingFileReader
from src.learning.training.training_transformer import TrainingTransformer
from src.learning.models import create_cnn, create_mlp, create_multi_model

In [ ]:
config_manager = ConfigurationManager()
config = config_manager.config

reader = TrainingFileReader(path_to_training='../../training/laps/')
transformer = TrainingTransformer(config)
collector = LabelCollector()

# axis=2 for frames, axis=0 for telem and diffs
def memory_creator(instance, memory, length=4, interval=2, axis=2):
    memory.append(instance)
    
    near_memory = memory[::-interval]
    if len(near_memory) < length:
        return None
    
    if len(memory) >= length * interval:
        memory.pop(0)
        
    return np.concatenate(near_memory, axis=axis)


def read_stored_data(filename):
    df = reader.read_telemetry_as_csv(filename + '.csv')
    telemetry = collector.collect_numeric_inputs(df)
    diffs = collector.collect_expert_labels(df)

    frames = reader.read_training_video(filename + '.avi')
    resized_frames_np = transformer.resize_video_for_training(frames)

    return resized_frames_np, telemetry, diffs


def prep_full_mem_datas(frames, telemetry, diffs, length, interval):    
    mem_slice_frames = []
    mem_slice_telemetry = []
    
    mem_frames = []
    for frame in frames:
        mem_frame = memory_creator(frame, mem_slice_frames, length=length, interval=interval, axis=2)

        if mem_frame is not None:
            mem_frames.append(mem_frame)
    mem_frames_np = np.array(mem_frames)
    
    mem_telemetry = []
    for index, telem in telemetry.iterrows():
        mem_telem = memory_creator(telem, mem_slice_telemetry, length=length, interval=interval, axis=0)

        if mem_telem is not None:
            mem_telemetry.append(mem_telem)
    mem_telemetry_np = np.array(mem_telemetry)
    
    mem_diffs = diffs
    len_diff = diffs.shape[0] - mem_frames_np.shape[0]
    if len_diff > 0:
        mem_diffs = mem_diffs.iloc[len_diff:]
    mem_diffs_np = mem_diffs.to_numpy()
    
    assert mem_frames_np.shape[0] == mem_telemetry_np.shape[0] == mem_diffs_np.shape[0], "Lengths differ!"
    return mem_frames_np, mem_telemetry_np, mem_diffs_np


def prep_mem_datas(frames, telemetry, diffs, length, interval):
    mem_telemetry = telemetry
    mem_diffs = diffs
    
    memory = []
    mem_frames = []
    for frame in frames:
        mem_frame = memory_creator(frame, memory, length=length, interval=interval, axis=2)

        if mem_frame is not None:
            mem_frames.append(mem_frame)

    mem_frames_np = np.array(mem_frames)

    len_diff = mem_telemetry.shape[0] - mem_frames_np.shape[0]
    if len_diff > 0:
        mem_telemetry = mem_telemetry.iloc[len_diff:]
        mem_diffs = mem_diffs.iloc[len_diff:]
    
    mem_telemetry_np = mem_telemetry.to_numpy()
    mem_diffs_np = mem_diffs.to_numpy()

    return mem_frames_np, mem_telemetry_np, mem_diffs_np


def plot_stuff(title, plot_elems, figsize=(18, 10)):
    fig=plt.figure(figsize=figsize)
    plt.title(title)
    #plt.ylabel('dunno')
    plt.xlabel('Epoch')
    x = np.arange(0, len(plot_elems[0]['data']), 1)
    
    for plot_elem in plot_elems:
        plt.errorbar(x, plot_elem['data'], yerr=plot_elem['error'], label=plot_elem['label'], alpha=plot_elem['alpha'], fmt='-o', capsize=5)

    plt.grid(axis='x')
    plt.legend(loc='best', prop={'size': 15})
    plt.show()
    plt.savefig('./' + title + '.png')

    
filenames = ['lap_5_2020_01_24', 'lap_6_2020_01_24', 'lap_7_2020_01_24']

In [ ]:
frames, telems, diffs = read_stored_data(filenames[0])
mem_frames, mem_telems, mem_diffs = prep_full_mem_datas(*read_stored_data(filenames[0]), 4, 1)

print(mem_frames.shape)
print(mem_telems.shape)
print(mem_diffs.shape)

In [ ]:
experiments = [(1, 1), (4, 1), (8, 1), (16, 1)]
datas_list = []

for experiment in tqdm(experiments):
    frames_mem = None
    telemetry_mem = None
    diffs_mem = None
    
    for training_file in filenames:
        frames_i, telem_i, diffs_i = prep_full_mem_datas(*read_stored_data(training_file), *experiment)
        
        if frames_mem is None and telemetry_mem is None and diffs_mem is None:
            frames_mem = frames_i
            telemetry_mem = telem_i
            diffs_mem = diffs_i
        else:
            frames_mem = np.append(frames_mem, frames_i, axis=0)
            telemetry_mem = np.append(telemetry_mem, telem_i, axis=0)
            diffs_mem = np.append(diffs_mem, diffs_i, axis=0)
        gc.collect()
        
    datas_list.append((frames_mem, telemetry_mem, diffs_mem))

In [ ]:
epochs = 20
batch_size = 32
verbose = 0

results = {}

for data in tqdm(datas_list):
    result_desc = "N={}, M={}".format(*experiments[datas_list.index(data)])
    tqdm.write(result_desc)
    
    losses = []
    val_losses = []
    for i in tqdm(range(0, 15)):
        frames_train, frames_test, telemetry_train, telemetry_test, diffs_train, diffs_test = train_test_split(*data, test_size=0.2)
        
        mlp = create_mlp(input_shape=telemetry_train[0].shape)
        cnn = create_cnn(input_shape=frames_train[0].shape)
        multi = create_multi_model(mlp, cnn)
    
        hist = multi.fit([telemetry_train, frames_train], diffs_train,
                        validation_data=([telemetry_test, frames_test], diffs_test),
                        epochs=epochs, batch_size=batch_size, verbose=verbose)
        
        losses.append(hist.history['loss'])
        val_losses.append(hist.history['val_loss'])
    
        gc.collect()
    
    mean_losses = [np.mean(i) for i in zip(*losses)]
    std_losses = [np.std(i) for i in zip(*losses)]
    mean_val_losses = [np.mean(i) for i in zip(*val_losses)]
    std_val_losses = [np.std(i) for i in zip(*val_losses)]
    
    results[result_desc] = mean_losses
    results[result_desc + ' std'] = std_losses
    results[result_desc + ' val'] = mean_val_losses
    results[result_desc + ' val std'] = std_val_losses
    
    tqdm.write("Mean loss per epoch: {}".format(mean_losses))
    tqdm.write("Mean validation loss per epoch: {}".format(mean_val_losses))
    
    gc.collect()

In [ ]:
loss_data = []
val_loss_data = []

for key, loss in results.items():
    if 'std' in key or 'raw' in key:
        continue
    
    if 'val' in key:
        val_loss_data.append({'data': loss, 'error': results[key + ' std'], 'label': key, 'alpha': 1.0})
    else:
        loss_data.append({'data': loss, 'error': results[key + ' std'], 'label': key, 'alpha': 1.0})
        
plot_stuff("val losses", val_loss_data, figsize=(10, 14))
plot_stuff("losses", loss_data, figsize=(10, 14))

In [ ]:
mlp = create_mlp()
cnn = create_cnn(input_shape=frames_train[0].shape)
multi = create_multi_model(mlp, cnn)
#multi.summary()

from keras.utils import plot_model
plot_model(multi, to_file='../../../Downloads/multi_model.png')